<a href="https://colab.research.google.com/github/jurtta66/tracking_synapse/blob/main/Kononov/basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используем fast.ai

Сетап

In [ ]:
! [ -e /content ] && pip install -Uqq fastbook nbdev
import fastbook
fastbook.setup_book()
from fastbook import *

Сперва преобразуем кадры с видеокамер в картинки людей.
Для этого пробежимся по кадрам, на каждом детектируем людей, для каждой детекции обрежем bbox из кадра в новое изображение.


Путь к папке с кадрами

In [ ]:
path = Path('/content/drive/MyDrive/comp_v/data/images/train')
path

In [2]:
from fastai.vision.all import *
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as T

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image

Скачаем предобученную модель для детекции

In [ ]:
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

Определим функции predict и отрисовки картинки кадра с bbox, 'вырезки' области из изображения

In [ ]:
def predict(img):
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    with torch.no_grad():
        pred = model([img])[0]
    return pred['labels'].detach().numpy(), pred['boxes'].detach().numpy(), pred['scores'].detach().numpy()

def show_img_with_bbox(img, bbox):
    fig, ax = plt.subplots()
    ax.imshow(img)
    x1, y1, x2, y2 = bbox
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='g', facecolor='none')
    ax.add_patch(rect)
    plt.show()

def show_img(img):
    fig, ax = plt.subplots()
    ax.imshow(img)
    plt.show()

def crop_img(img, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    return img.crop((x1, y1, x2, y2))

Закинем в папку вырезанные изображения людей.

In [ ]:
output_path = Path('/content/gdrive/MyDrive/cv/synapse/person_images')
str(output_path)

In [ ]:
i = 0
for img_path in path.ls():
    person_count = 0
    i += 1
    print(i, end = ', ')
    img = PILImage.create(img_path)
    labels, boxes, scores = predict(img)
    if i == 201:
        print('done 200, input?')
        input()
    for label, bbox, score in zip(labels, boxes, scores):
        if label == 1 and score > 0.5:
            person_count += 1
            #print(f'Person detected in {img_path.name} with probability {score:.04f}')
            #print(f'Bounding box: {bbox}')
            person = crop_img(img, bbox)
            person.save(str(output_path) + '/' + str(i) + '_' + str(person_count) + '.jpg', 'JPEG')
            #show_img_with_bbox(img, bbox)
            #show_img(person)
            #input()

In [ ]:
len(output_path.ls())

2355

Теперь пропустим этих людей через resnet с обрезанным последним слоем, и для каждого человека сохраним вектор значений предпоследнего слоя. (если бы сохраняли последний слой реснет - то получили бы аргумент для сигмоиды -> вероятность, что на картинке человек или собака или допустим чайник. Нам это не нужно, мы знаем что на картинке человек.  

In [ ]:
transform = T.Compose([
    T.Resize((50, 150)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
transform_to_pil = T.ToPILImage()

Загружаем обрезанные изображения людей

In [ ]:
images = []
for filename in os.listdir(str(output_path)):
    image = Image.open(os.path.join(str(output_path), filename))
    #show_img(image)
    #print(image.size)
    image = transform(image)

    #show_img(transform_to_pil(image))
    #print(transform_to_pil(image).size)
    #input()
    images.append(image)

In [ ]:
images[0].shape, len(images)

Теперь возьмем классификатор, обрежем выходной слой и пустим через него наши изображения, после чего возьмем значения нового выходного слоя.

In [ ]:
resnet = models.resnet18(pretrained=True)

# Удаление последнего слоя
modules = list(resnet.children())[:-1]
resnet = torch.nn.Sequential(*modules)

# Отключение обучения для всех слоев
for p in resnet.parameters():
    p.requires_grad = False

# Пропуск изображений через модель
features = []
for image in images:
    # Изменение размера изображения и добавление размерности пакета
    image = torch.unsqueeze(image, 0)
    # Пропуск изображения через модель
    feature = resnet(image)
    # Удаление размерности пакета и добавление в список признаков
    features.append(torch.squeeze(feature))

Сделаем тоже самое, но для 2355 изображений людей и специальной сетки re-id OSNet



In [ ]:
!pip install torchreid

In [ ]:
import torchreid

# Загрузка OSNet
osnet = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=1041, # это значение не имеет значения для извлечения признаков
    loss='softmax',
    pretrained=True
)

# Перевод модели в режим оценки
osnet.eval()

# Пропуск изображений через модель
'''
osnet_features = []
with torch.no_grad():
    for image in images:
        # Изменение размера изображения и добавление размерности пакета
        image = torch.unsqueeze(image, 0)
        # Пропуск изображения через модель
        feature = osnet(image)
        # Удаление размерности пакета и добавление в список признаков
        osnet_features.append(torch.squeeze(feature))
'''

In [ ]:
osnet_features_np = np.array([tensor.tolist() for tensor in osnet_features])

Сохраним osnet_features в текстовый файл

In [ ]:
#np.save('/content/drive/MyDrive/cv/synapse/osnet-features-vectors', osnet_features_np)
osnet_features_np = np.load('/content/gdrive/MyDrive/cv/synapse/osnet-features-vectors/osnet-features-vectors.npy')

In [ ]:
osnet_features_np.shape

Сократим размерности векторов признаков с (1, 512) до (1, 2) и (1, 3) с помощью t-sne.

In [ ]:
from sklearn.manifold import TSNE

# экземпляры t-sne
tsne2 = TSNE(n_components=2)
tsne3 = TSNE(n_components=3)

# Преобразование t-sne
osnet_2 = tsne2.fit_transform(osnet_features_np)
osnet_3 = tsne3.fit_transform(osnet_features_np)
#f_reduced_2 = tsne2.fit_transform(features_np)
#f_reduced_3 = tsne3.fit_transform(features_np)

Кластеризуем вектора размерности 2 (3). Я поглядел на первые 100 картинок, там 45- 60 людей. Хотя здесь 2355 людей, и если отношение такое же, нужна примерна половина кластеров, но я сделал 700. Так возможно похожие люди в один кластер попадут (подозреваю, стоит сделать еще меньше по этой же причине). А пока хочется именно одинакового человека на кластере отлавливать.

In [ ]:
from sklearn.cluster import KMeans

#n_clusters = 770  #
n_clusters = 24
kmeans = KMeans(n_clusters=n_clusters)

kmeans.fit_transform(osnet_clean_2)
cluster_labels = kmeans.labels_
cluster_labels

In [ ]:
len(cluster_labels)

Покажем на 2д графике кластеры

In [ ]:
def plot_clusters(feature_vectors, n_clusters, cluster_labels=cluster_labels):
    # Спектральная карта цветов
    colors = plt.cm.Spectral(np.linspace(0, 1, n_clusters))

    # График
    for i in range(len(feature_vectors)):
        point = feature_vectors[i]
        plt.scatter(point[0], point[1], c=colors[cluster_labels[i]])

    plt.show()

Теперь хотим посмотреть, насколько разрозненными получаются точки, соответствующие разным кластерам (насколько четко osnet плоттит людей). Для этого пробежимся по кластерам, найдем большие (size >= 3), запишем их в массив big_clusters.
Все это - чтобы создать чистый датасет, с 25 людьми, по 3-5 кадров на каждого.

In [ ]:
'''
def show_cluster_imgs(cluster_label, img_path, batch_size=10):
    # Цикл по директории, получаем список индексов людей одного кластера.
    cluster_indices = [i for i in range(len(osnet_2)) if cluster_labels[i] == cluster_label]
    # Цикл по папке людей, если индекс есть в cluster_indices -> добавляем в массив
    cluster_imgs = []
    print(len(cluster_indices))
    index = 0
    for img in img_path.ls():
        if index in cluster_indices:
            img = PILImage.create(img)
            #img = transform(img)
            cluster_imgs.append(img)
        index += 1
    print(len(cluster_imgs))

    for i in range(min(batch_size, len(cluster_imgs))):
        show_img(cluster_imgs[i])

#show_cluster_imgs(106, output_path)
'''
def calc_cluster(cluster_label, img_path):
    # out: python список pil potential изображений людей, принадлежащих одному кластеру

    # Цикл по директории, получаем список индексов людей одного кластера.
    cluster_indices = [i for i in range(len(osnet_2)) if cluster_labels[i] == cluster_label]

    # Цикл по папке людей, если индекс есть в cluster_indices -> добавляем в массив
    cluster_imgs = []
    index = 0
    for img in img_path.ls():
        if index in cluster_indices:
            img = PILImage.create(img)
            #img = transform(img)
            cluster_imgs.append(img)
        index += 1
    return cluster_imgs

def show_cluster(cluster_imgs, batch_size=10):
    # Показываем картинки одного кластера
    for i in range(min(batch_size, len(cluster_imgs))):
        img = PILImage.create(cluster_imgs[i])
        show_img(img)

def calc_big_clusters(bar=3, big_clusters_num=25):
    big_clusters = []  # Сюда записываем кластеры размера >=bar
    # Пробегаемся по меткам кластеров и добавляем большие кластера в список
    for i in range(n_clusters):
        cluster = calc_cluster(cluster_label=i, img_path=output_path)
        if len(cluster) >= bar:
            big_clusters.append((i, cluster))
        # Если наелись
        if len(big_clusters) == big_clusters_num:
            return big_clusters
    return ['did not reach num']

#big_clusters = calc_big_clusters()
#big_clusters_np = np.array(big_clusters)
#big_clusters_np.shape

In [ ]:
big_clusters_np = np.array([cluster[1] for cluster in big_clusters])
[len(cluster) for cluster in big_clusters_np]

Проверим на достоверность: покажем картинки людей с одного ббольшого кластера

In [ ]:
for cluster in big_clusters:
    print(f"cluster number {cluster[0]}")
    print(cluster[1])
    show_cluster(cluster[1])
    #for img in big_clusters[1]:
        #print(img)
        #show_img(img)
    input()

Я поглядел, из 25 кластеров один поломанный, штуки 4 не очень качественные. Прогоним людей из кластеров через osnet.

In [ ]:
noisy_clusters_indices = [7]  # поломанный кластер
big_clusters = [pair for pair in big_clusters if pair[0] not in noisy_clusters_indices]

In [ ]:
osnet_clean = []  # векторы признаков для новых изобраежений
with torch.no_grad():
    i =0
    # перебираем кластеры
    for cluster in big_clusters:
        print(i)
        i+=1
        label, imgs = cluster
        for img in imgs:
            img_tensor = transform(PILImage.create(img))
            img_tensor = torch.unsqueeze(img_tensor, 0)
            vector = osnet(img_tensor)
            osnet_clean.append(torch.squeeze(vector))

In [ ]:
osnet_clean_np = np.array([tensor.tolist() for tensor in osnet_clean])
osnet_clean_np.shape
#osnet_clean_np[0].shape

In [ ]:
# Преобразование t-sne
osnet_clean_2 = tsne2.fit_transform(osnet_clean_np)

Построим график. Для 25 людей получается не прям плохая картинка.

In [ ]:
#osnet_clean_2[0][1]
plot_clusters(feature_vectors=osnet_clean_2, n_clusters=24)

Посчитаем значение метрики map@K для алгоритма Re-id на основе евкл растояния. Для этого используем датасет с помеченными людьми.

Загрузка изображений

In [23]:
labeled_people_path = Path('/content/gdrive/MyDrive/cv/synapse/whole_body_images')

In [24]:
def get_id(filename):
    return int(filename[:3])

In [25]:
labeled_images = []
labels = []
# Проходимся по директории
for img_path in labeled_people_path.ls():
    labels.append(get_id(img_path.name))
    img = PILImage.create(img_path)
    #img = transform(img)
    labeled_images.append(img)

In [ ]:
labeled_images[200].size

Теперь пропустим изображения через osnet

In [ ]:
osnet_mapk = []
i = 0
for img in labeled_images:
    img_tensor = torch.unsqueeze(transform(img), 0)
    vector = osnet(img_tensor)
    osnet_mapk.append(torch.squeeze(vector))

сохраним/ загрузим osnet_mapk

In [3]:
#osnet_mapk_np = np.array([tensor.tolist() for tensor in osnet_mapk])
#np.save('/content/gdrive/MyDrive/cv/synapse/osnet-mapk', osnet_mapk_np)
osnet_mapk = np.load('/content/gdrive/MyDrive/cv/synapse/osnet-mapk.npy')

алгоритм re-id

In [29]:
def reid_euclid(v, vectors, n=5):
    # in: вектор, список векторов (вкл этот вектор)
    # out: первые n самых близких по евклидовой метрике векторов к v

    distances = [np.linalg.norm(np.array(v) - np.array(vector)) for vector in vectors]
    closest_indices = np.argsort(distances)[1:n+1]
    return closest_indices.tolist()

reid_euclid([0, 0], [[0, 0], [1, 1], [2, 2], [0, 1], [-1, 19]], n=2)

[3, 1]

Организуем вычисление map@k для нашего проаннотированного датасета.

In [30]:
def pk(img_index, objs, r_true, K=5):
    # in: индекс изображения, по которому вычисляем pk, изображения, аннотации, K.
    # out: значение p@k
    preds = reid_euclid(objs[img_index], objs, n=K)
    #print(preds)
    return sum([r_true[img_index]==r_true[i] for i in preds])/K
    #return preds
    #return sum([r_true[img_index] == reid_euclid(objs[img_index], objs, n=5)[img_index] for img_index in range(len(objs))])/K

#for index in [2, 114, 212, 192, 284]:
    #labeled_images[index].show()


In [31]:
def apk(img_index, objs, r_true, K=5):
    preds = reid_euclid(objs[img_index], objs, n=K)
    weighted_pk = [(r_true[img_index]==r_true[preds[i]]) * pk(img_index, objs, r_true, i+1) for i in range(len(preds))]
    return sum(weighted_pk)/K

# тест
test_arr = [[0, 1], [3, 1], [2, 0], [3, 3], [1, 1], [0, 0.5], [5, 3], [3, 2]]
test_labels = [0, 1, 0, 2, 0, 0, 2, 1]
apk(1, test_arr, test_labels, K=3)

0.3333333333333333

In [38]:
def mapk(objs, r_true, K=5):
    N = len(objs)
    apk_sum = 0
    for img_index in range(N):
        apk_sum += apk(img_index, objs, r_true, K)
    return (apk_sum)/N
#mapk(test_arr, test_labels, K=3)
mapk(osnet_mapk, labels, K=5)

0.3124000000000003

в датасете было по 5 фоток каждого человека. map@k получился плоховатый, возможно что-то в алгоритме напутал (при написании тестировал на простых векторах, вроде работало). map@k макс при K=5, то есть условно у алгоритма есть 5 взвешенных попыток угадать 4 фото из датасета, (не считая самого рассматриваемого изображения)


